In [1]:
import re
import pandas as pd

In [2]:
def guess_column_types(file_path, delimiter=',', has_headers=True):
    try:
        # Read the CSV file using the specified delimiter and header settings
        df = pd.read_csv(file_path, sep=delimiter,low_memory=False, header=0 if has_headers else None)

        # Initialize a dictionary to store column data types
        column_types = {}

        # Loop through columns and infer data types
        for column in df.columns:
            # sample_values = df[column].dropna().sample(min(5, len(df[column])), random_state=42)

            # Check for datetime format "YYYY-MM-DD HH:MM:SS"
            is_datetime = all(re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(value)) for value in df[column])

            # Check for date format "YYYY-MM-DD"
            is_date = all(re.match(r'\d{4}-\d{2}-\d{2}', str(value)) for value in df[column])

            # Assign data type based on format detection
            if is_datetime:
                inferred_type = 'datetime64'
            elif is_date:
                inferred_type = 'date'
            else:
                inferred_type = pd.api.types.infer_dtype(df[column], skipna=True)

            column_types[column] = inferred_type

        return (True, column_types)  # Return success and column types
    except pd.errors.ParserError:
        return (False, str(e))  # Return error message

In [3]:
file_path = '../data/LoanStats_web.csv'
result, column_types_or_error = guess_column_types(file_path)

if result:
    print("Column Types:", column_types_or_error)
else:
    print("Error:", column_types_or_error)


Column Types: {'id': 'string', 'member_id': 'floating', 'loan_amnt': 'floating', 'funded_amnt': 'floating', 'funded_amnt_inv': 'floating', 'term': 'string', 'int_rate': 'string', 'installment': 'floating', 'grade': 'string', 'sub_grade': 'string', 'emp_title': 'string', 'emp_length': 'string', 'home_ownership': 'string', 'annual_inc': 'floating', 'verification_status': 'string', 'issue_d': 'string', 'loan_status': 'string', 'pymnt_plan': 'string', 'url': 'floating', 'desc': 'string', 'purpose': 'string', 'title': 'string', 'zip_code': 'string', 'addr_state': 'string', 'dti': 'floating', 'delinq_2yrs': 'floating', 'earliest_cr_line': 'string', 'inq_last_6mths': 'floating', 'mths_since_last_delinq': 'floating', 'mths_since_last_record': 'floating', 'open_acc': 'floating', 'pub_rec': 'floating', 'revol_bal': 'floating', 'revol_util': 'string', 'total_acc': 'floating', 'initial_list_status': 'string', 'out_prncp': 'floating', 'out_prncp_inv': 'floating', 'total_pymnt': 'floating', 'total_p

In [4]:
column_types_corrected = {col: ('float64' if t == 'floating' else t) for col, t in column_types_or_error.items()}

# สำหรับ 'date' และ 'datetime', pandas รู้จัก 'datetime64'
column_types_corrected = {col: ('datetime64' if t == 'date' else 'float64' if t == 'floating' else t) for col, t in column_types_corrected.items()}


# Assuming `data.csv` is your file name
raw_df = pd.read_csv('../data/LoanStats_web.csv', dtype=column_types_corrected)

In [5]:
catFeature = 'purpose'
contTarget = 'loan_amnt'

In [6]:
# Calculate the percentage of missing values for each column
missing_percentage = raw_df.isnull().mean() * 100

# Filter out columns where more than 30% of the data is missing
columns_to_keep = missing_percentage[missing_percentage <= 30].index.tolist()
filtered_df = raw_df[columns_to_keep]

In [7]:
raw_df = filtered_df

In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432466 entries, 0 to 1432465
Data columns (total 100 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   loan_amnt                   1432440 non-null  float64
 1   funded_amnt                 1432440 non-null  float64
 2   funded_amnt_inv             1432440 non-null  float64
 3   term                        1432440 non-null  string 
 4   int_rate                    1432440 non-null  string 
 5   installment                 1432440 non-null  float64
 6   grade                       1432440 non-null  string 
 7   sub_grade                   1432440 non-null  string 
 8   emp_title                   1302202 non-null  string 
 9   emp_length                  1323970 non-null  string 
 10  home_ownership              1432440 non-null  string 
 11  annual_inc                  1432440 non-null  float64
 12  verification_status         1432440 non-null  string 
 

### จัดเตรียมข้อมูลแบบ Series เพื่อการทดสอบความสัมพันธ์

In [9]:
import pandas as pd
import numpy as np
from scipy.stats import kruskal
from scikit_posthocs import posthoc_dunn

# สมมติว่า df_clean เป็น DataFrame ที่คุณมีอยู่และ 'catFeature' เป็นคอลัมน์ที่มีค่า categorical หลาย ค่า
# สร้างสำเนาเพื่อหลีกเลี่ยง SettingWithCopyWarning
df_clean = raw_df.copy()

# แปลงข้อมูลใน contTarget ให้เป็นตัวเลข
## df_clean[contTarget] = pd.to_numeric(df_clean[contTarget], errors='coerce')

# ตรวจสอบและลบค่า NaN หลังจากการแปลง
df_clean = df_clean.dropna(subset=[contTarget])

# หลังจากแปลงข้อมูลแล้ว สร้าง groups ซึ่งประกอบด้วย series ของ loan_amnt หลายๆ ชุด ที่ถูกสร้างขึ้นจาก value แต่ละแบบของ catFeature
groups = [df_clean[df_clean[catFeature] == value][contTarget] for value in df_clean[catFeature].unique()]

# ลบ Series ที่ว่างเปล่า (ถ้ามี)
groups = [group for group in groups if not group.empty]

/opt/homebrew/Caskroom/miniforge/base/envs/python3.8/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Kolmogorov-Smirnov Test และ Anderson-Darling Test เพื่อดูว่ามีการแจกแจงแบบปกติ หรือไม่

### Mann-Whitney U test หรือ Kruskal-Wallis H-test กรณีตั้งแต่ 3 กลุ่ม

In [10]:
# ทำ Kruskal-Wallis H-test
stat, p = kruskal(*groups)
print(f'Kruskal-Wallis H-test stat={stat:.3f}, p={p:.3e}')

# อ่านค่าและการแปรผล
alpha = 0.05  # ระดับนัยสำคัญ
if p < alpha:
    print("มีความแตกต่างที่มีนัยสำคัญทางสถิติระหว่างกลุ่ม (reject H0)")
else:
    print("ไม่มีความแตกต่างที่มีนัยสำคัญทางสถิติระหว่างกลุ่ม (fail to reject H0)")

Kruskal-Wallis H-test stat=80061.420, p=0.000e+00
มีความแตกต่างที่มีนัยสำคัญทางสถิติระหว่างกลุ่ม (reject H0)


In [23]:
total_records = len(raw_df)

In [24]:
total_records

1432466

In [22]:
# ตรวจสอบจำนวนเรกคอร์ดทั้งหมดใน DataFrame
total_records = len(raw_df)

# สร้างรายการของคอลัมน์ที่มีข้อมูลครบถ้วน (ไม่มีค่า null)
complete_columns = [column for column in raw_df.columns if raw_df[column].isnull().sum() == 0]

# ตรวจสอบว่ามีคอลัมน์ใดบ้างที่มีข้อมูลครบถ้วน
if not complete_columns:
    print("ไม่มีคอลัมน์ใดที่มีข้อมูลครบถ้วน")
else:
    # เลือกเฉพาะคอลัมน์ที่มีข้อมูลครบถ้วนจาก raw_df
    df_complete = raw_df[complete_columns]

    # แสดงผลลัพธ์
    print("คอลัมน์ที่มีข้อมูลครบถ้วน:", complete_columns)
    df_complete.info()


ไม่มีคอลัมน์ใดที่มีข้อมูลครบถ้วน


In [27]:
# สร้างรายการของคอลัมน์ที่มี Non-Null จำนวน 1,432,440
selected_columns = [col for col in raw_df.columns if raw_df[col].notnull().sum() == 1432440]

# แสดงคอลัมน์ที่เลือก
print("Selected columns:", selected_columns)

# สร้าง DataFrame ใหม่จากคอลัมน์ที่เลือก
df_selected = raw_df[selected_columns]


# ลบแถวที่มีค่า null ในคอลัมน์เหล่านั้น
df_cleaned = df_selected.dropna()

# แสดงข้อมูลทั่วไปของ DataFrame หลังจากลบ null
df_cleaned.info()

Selected columns: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'addr_state', 'delinq_2yrs', 'earliest_cr_line', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_30dpd', 

In [28]:
import sweetviz as sv
import pandas as pd

# Load your data into a Pandas DataFrame
### df = pd.read_csv("your_data.csv")

# Create an analysis report for your data
report = sv.analyze(df_cleaned)

# Display the report
report.show_html()

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
